# AgentCore 메모리를 활용한 Strands Agents (단기 메모리)


## 소개

이 튜토리얼은 AgentCore **단기 메모리** (Raw events)를 사용하여 Strands agents로 **개인 에이전트**를 구축하는 방법을 보여줍니다. 

에이전트는 `get_last_k_turns`를 사용하여 세션의 최근 대화를 기억하고 사용자가 돌아왔을 때 대화를 원활하게 이어갈 수 있습니다.


### 튜토리얼 세부사항

| 정보                | 세부사항                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| 튜토리얼 유형       | 단기 대화형                                                                       |
| 에이전트 유형       | 개인 에이전트                                                                     |
| 에이전트 프레임워크 | Strands Agents                                                                   |
| LLM 모델           | Amazon Nove Pro                                                    |
| 튜토리얼 구성요소   | AgentCore 단기 메모리, AgentInitializedEvent 및 MessageAddedEvent 훅            |
| 예제 복잡도        | 초급                                                                             |

다음을 배우게 됩니다:
- 대화 연속성을 위한 단기 메모리 사용
- 최근 K개 대화 턴 검색
- 실시간 정보를 위한 웹 검색 도구
- 대화 기록으로 에이전트 초기화

## 사전 요구사항

- Python 3.10+
- AgentCore Memory 권한이 있는 AWS 자격 증명
- AgentCore Memory 역할 ARN
- Amazon Bedrock 모델 액세스

환경 설정을 시작해보겠습니다!

## 1단계: 설정 및 가져오기

In [ ]:
!pip install -qr requirements.txt

In [ ]:
# 로깅을 설정합니다.

import logging
from datetime import datetime

# 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("personal-agent")

In [ ]:
# 모델 설정

import boto3

region = boto3.session.Session().region_name

NOVA_PRO_MODEL_ID = "us.amazon.nova-pro-v1:0"
if region.startswith("eu"):
    NOVA_PRO_MODEL_ID = "eu.amazon.nova-pro-v1:0"
elif region.startswith("ap"):
    NOVA_PRO_MODEL_ID = "apac.amazon.nova-pro-v1:0"

print(f"Nova Pro Model ID: {NOVA_PRO_MODEL_ID}")

In [ ]:
# 패키지 정보 및 구성 정보 가져오기
import os
from strands import Agent, tool
from strands.hooks import AgentInitializedEvent, HookProvider, HookRegistry, MessageAddedEvent
from bedrock_agentcore.memory import MemoryClient

# 구성
REGION = os.getenv('AWS_REGION', 'us-west-2') # 에이전트를 위한 AWS 리전
ACTOR_ID = "user_123" # 고유 식별자 (AgentID, User ID 등)
SESSION_ID = "personal_session_001" # 고유 세션 식별자


## 2단계: 웹 검색 도구

먼저 에이전트를 위한 간단한 웹 검색 도구를 만들어보겠습니다.

In [ ]:
from ddgs.exceptions import DDGSException, RatelimitException
from ddgs import DDGS

# strands sdk 를 이용하여 도구를 만듭니다. 
@tool
def websearch(keywords: str, region: str = "us-en", max_results: int = 5) -> str:
    """최신 정보를 위해 웹을 검색합니다.
    
    Args:
        keywords (str): 검색 쿼리 키워드.
        region (str): 검색 지역: wt-wt, us-en, uk-en, ru-ru 등..
        max_results (int | None): 반환할 최대 결과 수.
    Returns:
        검색 결과가 포함된 딕셔너리 목록.
    
    """
    try:
        results = DDGS().text(keywords, region=region, max_results=max_results)
        return results if results else "결과를 찾을 수 없습니다."
    except RatelimitException:
        return "속도 제한에 도달했습니다. 나중에 다시 시도해주세요."
    except DDGSException as e:
        return f"검색 오류: {e}"
    except Exception as e:
        return f"검색 오류: {str(e)}"

logger.info("✅ 웹 검색 도구 준비 완료")

## 3단계: 메모리 리소스 생성

단기 메모리의 경우, 전략 없이 메모리 리소스를 생성합니다. 

이는 `get_last_k_turns`로 검색할 수 있는 원시 대화 턴을 저장합니다. 

`get_last_k_turns` 는 최근 K개의 대화 턴(turn)만 조회하는 헬퍼 함수입니다. 


In [ ]:
from botocore.exceptions import ClientError

# 메모리 클라이언트 초기화
client = MemoryClient(region_name=REGION)
memory_name = "PersonalAgentMemory"

try:
    # 전략 없이 메모리 리소스 생성 (따라서 단기 메모리에만 액세스)
    memory = client.create_memory_and_wait(
        name=memory_name,
        strategies=[],  # 단기 메모리를 위한 전략 없음
        description="개인 에이전트를 위한 단기 메모리",
        event_expiry_days=7, # 단기 메모리 보존 기간. 최대 365일까지 가능.
    )
    memory_id = memory['id']
    logger.info(f"✅ 메모리 생성됨: {memory_id}")
except ClientError as e:
    logger.info(f"❌ 오류: {e}")
    if e.response['Error']['Code'] == 'ValidationException' and "already exists" in str(e):
        # 메모리가 이미 존재하는 경우, ID 검색
        memories = client.list_memories()
        memory_id = next((m['id'] for m in memories if m['id'].startswith(memory_name)), None)
        logger.info(f"메모리가 이미 존재합니다. 기존 메모리 ID 사용: {memory_id}")
except Exception as e:
    # 메모리 생성 중 오류 표시
    logger.error(f"❌ 오류: {e}")
    import traceback
    traceback.print_exc()
    # 오류 시 정리 - 부분적으로 생성된 메모리 삭제
    if memory_id:
        try:
            client.delete_memory_and_wait(memory_id=memory_id)
            logger.info(f"메모리 정리됨: {memory_id}")
        except Exception as cleanup_error:
            logger.error(f"메모리 정리 실패: {cleanup_error}")

## 4단계: 메모리 훅

이 단계에서는 메모리 작업을 자동화하는 사용자 정의 `MemoryHookProvider` 클래스를 정의합니다. 

hook 은 에이전트 실행 생명주기의 특정 지점에서 실행되는 특수 함수입니다. 

우리가 만드는 메모리 훅은 두 가지 주요 기능을 제공합니다:

1. **최근 대화 로드**: `AgentInitializedEvent` 훅을 사용하여 에이전트가 초기화될 때 최근 대화 기록을 자동으로 로드합니다.
2. **마지막 메시지 저장**: 새로운 대화 메시지를 저장합니다.

이는 수동 관리 없이 원활한 메모리 경험을 만듭니다.

In [ ]:
class MemoryHookProvider(HookProvider):
    def __init__(self, memory_client: MemoryClient, memory_id: str):
        self.memory_client = memory_client
        self.memory_id = memory_id
    
    def on_agent_initialized(self, event: AgentInitializedEvent):
        """에이전트 시작 시 최근 대화 기록 로드"""
        try:
            # 에이전트 상태에서 세션 정보 가져오기
            actor_id = event.agent.state.get("actor_id")
            session_id = event.agent.state.get("session_id")
            
            if not actor_id or not session_id:
                logger.warning("에이전트 상태에 actor_id 또는 session_id가 없습니다")
                return
            
            # 메모리에서 최근 5개 대화 턴 로드
            recent_turns = self.memory_client.get_last_k_turns(
                memory_id=self.memory_id,
                actor_id=actor_id,
                session_id=session_id,
                k=5
            )
            
            if recent_turns:
                # 컨텍스트를 위한 대화 기록 형식화
                context_messages = []
                for turn in recent_turns:
                    for message in turn:
                        role = message['role']
                        content = message['content']['text']
                        context_messages.append(f"{role}: {content}")
                
                context = "\n".join(context_messages)
                # 에이전트의 시스템 프롬프트에 컨텍스트 추가.
                event.agent.system_prompt += f"\n\n최근 대화:\n{context}"
                logger.info(f"✅ {len(recent_turns)}개 대화 턴 로드됨")
                
        except Exception as e:
            logger.error(f"메모리 로드 오류: {e}")
    
    def on_message_added(self, event: MessageAddedEvent):
        """메모리에 메시지 저장"""
        messages = event.agent.messages
        try:
            # 에이전트 상태에서 세션 정보 가져오기
            actor_id = event.agent.state.get("actor_id")
            session_id = event.agent.state.get("session_id")

            if messages[-1]["content"][0].get("text"):
                self.memory_client.create_event(
                    memory_id=self.memory_id,
                    actor_id=actor_id,
                    session_id=session_id,
                    messages=[(messages[-1]["content"][0]["text"], messages[-1]["role"])]
                )
        except Exception as e:
            logger.error(f"메모리 저장 오류: {e}")
    
    def register_hooks(self, registry: HookRegistry):
        # 메모리 훅 등록
        registry.add_callback(MessageAddedEvent, self.on_message_added)
        registry.add_callback(AgentInitializedEvent, self.on_agent_initialized)

## 5단계: 웹 검색 기능을 갖춘 개인 에이전트 생성

In [ ]:
def create_personal_agent():
    """메모리와 웹 검색 기능을 갖춘 개인 에이전트 생성"""
    agent = Agent(
        name="PersonalAssistant",
        model= NOVA_PRO_MODEL_ID,
        system_prompt=f"""당신은 웹 검색 기능을 갖춘 도움이 되는 개인 어시스턴트입니다.
        
        다음과 같은 도움을 제공할 수 있습니다:
        - 일반적인 질문 및 정보 검색
        - 최신 정보를 위한 웹 검색
        - 개인 업무 관리
        
        최신 정보가 필요할 때는 websearch 함수를 사용하세요.
        오늘 날짜: {datetime.today().strftime('%Y-%m-%d')}
        친근하고 전문적으로 대응하세요.""",
        hooks=[MemoryHookProvider(client, memory_id)],
        tools=[websearch],
        state={"actor_id": ACTOR_ID, "session_id": SESSION_ID}
    )
    return agent

# 에이전트 생성
agent = create_personal_agent()
logger.info("✅ 메모리와 웹 검색 기능을 갖춘 개인 에이전트 생성됨")

#### 축하합니다! 에이전트가 준비되었습니다! :) 
## 에이전트를 테스트해보겠습니다

In [ ]:
# 메모리를 사용한 대화 테스트
print("=== 첫 번째 대화 ===")
print(f"사용자: 제 이름은 Alex이고 AI에 대해 배우는 것에 관심이 있습니다.")
print(f"에이전트: ", end="")
agent("제 이름은 Alex이고 AI에 대해 배우는 것에 관심이 있습니다.")

In [ ]:
print(f"사용자: 2025년 최신 AI 트렌드를 검색해주실 수 있나요?")
print(f"에이전트: ", end="")
agent("2025년 최신 AI 트렌드를 검색해주실 수 있나요?")

In [ ]:
print(f"사용자: 특히 머신러닝 응용 분야에 관심이 있습니다.")
print(f"에이전트: ", end="")
agent("특히 머신러닝 응용 분야에 관심이 있습니다.")

## 메모리 연속성 테스트

메모리 시스템이 올바르게 작동하는지 테스트하기 위해 에이전트의 새 인스턴스를 생성하고 이전에 저장된 정보에 액세스할 수 있는지 확인해보겠습니다:

아래에서 새로운 에이전트를 생성하지만 MemoryHookProvider 를 이용하여 만들기 때문에 hook 에 의해 같은 메모리를 참조할 수 있습니다. 

In [ ]:
# 새 에이전트 인스턴스 생성 (사용자 복귀 시뮬레이션)
print("=== 사용자 복귀 - 새 세션 ===")
new_agent = create_personal_agent()

# 메모리 연속성 테스트
print(f"사용자: 제 이름이 뭐였죠?")
print(f"에이전트: ", end="")
new_agent("제 이름이 뭐였죠?")

print(f"사용자: 머신러닝에 대한 더 많은 정보를 검색해주실 수 있나요?")
print(f"에이전트: ", end="")
new_agent("머신러닝에 대한 더 많은 정보를 검색해주실 수 있나요?")

## 저장된 메모리 보기

In [ ]:
# 메모리에 저장된 내용 확인
print("=== 메모리 내용 ===")
recent_turns = client.get_last_k_turns(
    memory_id=memory_id,
    actor_id=ACTOR_ID,
    session_id=SESSION_ID,
    k=3 # k를 조정하여 더 많거나 적은 턴을 확인
)

for i, turn in enumerate(recent_turns, 1):
    print(f"턴 {i}:")
    for message in turn:
        role = message['role']
        content = message['content']['text'][:100] + "..." if len(message['content']['text']) > 100 else message['content']['text']
        print(f"  {role}: {content}")
    print()

## 요약

이 튜토리얼에서는 개인 에이전트를 구축하는 방법을 보여주었습니다. 다음을 배웠습니다:

- 전략 없이 메모리 리소스 생성
- 대화 기록을 위한 `get_last_k_turns` 사용
- 에이전트에 웹 검색 기능 추가
- 컨텍스트 로딩을 위한 메모리 훅 구현

**다음 단계:**
- 더 정교한 도구 추가
- 장기 메모리 전략 구현
- 여러 소스를 활용한 검색 기능 향상

## 정리 (선택사항)

In [ ]:
# 메모리 리소스를 삭제하려면 주석 해제
# client.delete_memory_and_wait(memory_id)
# logger.info(f"✅ 메모리 삭제됨: {memory_id}")